In [ ]:
# change these to try this notebook out
BUCKET = 'gcp_model1'
PROJECT = 'cp100'
REGION = 'us-east1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [9]:
from keras import models
from keras import layers
import tensorflow as tf   

In [2]:
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 4us/step


In [3]:

mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

x_test -= mean
x_test /= std


In [4]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))

    model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])
    return model

In [5]:
model = build_model()
model.summary()

W0815 13:28:25.990439 30668 deprecation_wrapper.py:119] From C:\Users\Jumper.ai\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 13:28:26.027776 30668 deprecation_wrapper.py:119] From C:\Users\Jumper.ai\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 13:28:26.036725 30668 deprecation_wrapper.py:119] From C:\Users\Jumper.ai\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 13:28:26.076596 30668 deprecation_wrapper.py:119] From C:\Users\Jumper.ai\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.comp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                896       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(x_train, y_train, epochs=80, batch_size=16, verbose=0)

W0815 13:28:26.895393 30668 deprecation_wrapper.py:119] From C:\Users\Jumper.ai\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0815 13:28:26.899957 30668 deprecation_wrapper.py:119] From C:\Users\Jumper.ai\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



In [8]:
print("Test DAta:")
print(x_test[:1])
print("Expected Result:",model.predict(x_test[:1]))

Test DAta:
[[ 1.55369355 -0.48361547  1.0283258  -0.25683275  1.03838067  0.23545815
   1.11048828 -0.93976936  1.67588577  1.5652875   0.78447637 -3.48459553
   2.25092074]]
Expected Result: [[6.7123523]]


Deployment

In [ ]:
JOB_DIR="gs://%s/keras-job-dir" % BUCKET

In [ ]:
# Export the model to a SavedModel directory in Cloud Storage
export_path = tf.keras.experimental.export_saved_model(model, JOB_DIR + '/keras_export')
print("Model exported to: ", export_path)